In [ ]:
from spatialSSL.Dataloader import FullImageDatasetConstructor
from spatialSSL.Utils import split_dataset
from spatialSSL.Training import train

from spatialSSL.Training import train_epoch
from spatialSSL.Testing import test
from spatialSSL.Dataset import InMemoryGraphDataset
import numpy as np
import numpy as np
import matplotlib.pyplot as plt
from collections import Counter
from matplotlib.ticker import FuncFormatter
import zipfile
import torch
from spatialSSL.Dataloader import FullImageDatasetConstructor
from spatialSSL.Utils import split_dataset
from spatialSSL.Training import train, train_classification
from spatialSSL.Models import *
from spatialSSL.Training import train_epoch
from spatialSSL.Testing import test
from spatialSSL.Dataset import InMemoryGraphDataset
import numpy as np
from spatialSSL.Dataloader import FullImageDatasetConstructor
from spatialSSL.Utils import split_dataset,visualize_cell_type_accuracies
from spatialSSL.Training import train,train_classification,test_classification
from spatialSSL.Models import GAT4,Transferlearn
from spatialSSL.Training import train_epoch
from spatialSSL.Testing import test
from spatialSSL.Dataset import InMemoryGraphDataset
import torch.nn as nn
import numpy as np

In [ ]:
# encode book
category_encoding = {
    'Astro-Epen': 0,
    'CB GABA': 1,
    'CB Glut': 2,
    'CGE GABA': 3,
    'CNU GABA': 4,
    'CNU-HYa GABA': 5,
    'CNU-HYa Glut': 6,
    'HY GABA': 7,
    'HY Glut': 8,
    'HY Gnrh1 Glut': 9,
    'HY MM Glut': 10,
    'IT-ET Glut': 11,
    'Immune': 12,
    'LQ': 13,
    'LSX GABA': 14,
    'MB Dopa': 15,
    'MB GABA': 16,
    'MB Glut': 17,
    'MB-HB Sero': 18,
    'MGE GABA': 19,
    'MH-LH Glut': 20,
    'MOB-CR Glut': 21,
    'MOB-DG-IMN': 22,
    'MY GABA': 23,
    'MY Glut': 24,
    'NP-CT-L6b Glut': 25,
    'OEG': 26,
    'Oligo': 27,
    'P GABA': 28,
    'P Glut': 29,
    'Pineal Glut': 30,
    'TH Glut': 31,
    'Vascular': 32
}
import torch
def convert_to_float32(data_list):
    for data in data_list:
        # Convert attributes to float32
        data.x = data.x.to(dtype=torch.float32)
    return data_list



def thousands(x, pos):
    return f'{int(x)}k'

formatter = FuncFormatter(thousands)

def plot_histogram(graph_list, title, cell_num_path, cell_type_path):
    # Extracting number of cells in x
    num_cells = [data.x.size(0) / 1000 for data in graph_list] # Divide by 1000 to represent in k

    # Plotting histogram for number of cells in x
    plt.figure(figsize=(10, 5))
    plt.hist(num_cells, bins=30, edgecolor='black')
    plt.title(f'Number of Cells in Images for {title}')
    plt.xlabel('Number of Cells')
    plt.ylabel('Number of Images')
    plt.yticks(np.arange(0, max(plt.yticks()[0]), 1)) # Set y-ticks to integer values
    plt.gca().xaxis.set_major_formatter(formatter) # Apply the formatter
    plt.savefig(cell_num_path) # Save the plot
    plt.show()

    # Extracting cell types
    cell_types = [data.cell_type.tolist() for data in graph_list]
    flat_cell_types = [item for sublist in cell_types for item in sublist]

    # Get the encode_book from the first data object in the graph_list
    encode_book = graph_list[0].encode_book

    # Count the frequency of each cell type
    cell_type_counts = Counter(flat_cell_types)

    # Sort cell types by frequency
    sorted_cell_types = sorted(cell_type_counts.items(), key=lambda x: x[1]/1000, reverse=True) # Divide by 1000 to represent in k
    sorted_labels = [list(encode_book.keys())[cell_type[0]] for cell_type in sorted_cell_types] # Convert keys to list
    sorted_values = [cell_type[1] for cell_type in sorted_cell_types]

    # Plotting histogram for distribution of cell types
    plt.figure(figsize=(15, 5))
    plt.bar(range(len(sorted_values)), sorted_values, edgecolor='black')
    plt.title(f'Distribution of Cell Types for {title}')
    plt.xlabel('Cell Type')
    plt.ylabel('Number of Cells')
    plt.gca().yaxis.set_major_formatter(formatter) # Apply the formatter
    plt.xticks(ticks=range(len(sorted_labels)), labels=sorted_labels, rotation=90)
    plt.savefig(cell_type_path) # Save the plot
    plt.show()
    
def save_to_zip(loader, filename):
    with open(filename, 'wb') as f:
        pickle.dump(loader, f)
        
# Define a function to load the data from the ZIP file
def load_from_zip(zip_path, file_name):
    with zipfile.ZipFile(zip_path, 'r') as zipf:
        with zipf.open(file_name) as file:
            return torch.load(file)
        

In [ ]:
# Create an instance of Full_image_dataloader

 
#file_path = "./data/img_119670929.h5ad"
file_path = "./data/subset_6img_atlas_brain.h5ad"
#file_path = "./data/atlas_brain_638850_CCF.h5ad"
#file_path = "./data/sub_20img_adata.h5ad"

data_constracter = FullImageDatasetConstructor(file_path=file_path,
                                        image_col="section",
                                        label_col="class_label",
                                        include_label=False,
                                        radius=30,
                                        node_level = 1,
                                       random_mask_percentage = 0.1,
                                        encode_book = category_encoding
                                       #    niche_to_mask = 100
                                       #celltype_to_mask = "LQ"
                                               )


In [ ]:
# Load the data
data_constracter.load_data()
# Construct the graph
graph_list = data_constracter.construct_graph()
graph_list = convert_to_float32(graph_list)


In [ ]:
from sklearn.model_selection import train_test_split

# Split the graph_list into 80% for pre-training and 20% for pre-training validation
pre_train_list, pre_val_list = train_test_split(graph_list, test_size=0.20, random_state=42)


In [ ]:
# Plot histograms for pre_train_list and pre_val_list
plot_histogram(pre_train_list, 'Pre-Train', './processed_data/plots/pre_train_cell_num_img6_r30_n1_random_01.jpg', './processed_data/plots/pre_train_cell_type_img6_r30_n1_random_01.jpg')
plot_histogram(pre_val_list, 'Pre-Validation', './processed_data/plots/pre_val_cell_num_img6_r30_n1_random_01.jpg', './processed_data/plots/pre_val_cell_type_img6_r30_n1_random_01.jpg')


In [ ]:
# Save train_loader and val_loader to pickle files
torch.save(pre_train_list, './processed_data/pre_train_list.pt')
torch.save(pre_val_list, './processed_data/pre_val_list.pt')

# Create a ZIP file and add the pickle files to it
with zipfile.ZipFile('./processed_data/pre_training_data_img6_r30_n1_random_01.zip', 'w') as zipf:
    zipf.write('./processed_data/pre_train_list.pt', arcname='pre_train_list.pt')
    zipf.write('./processed_data/pre_val_list.pt', arcname='pre_val_list.pt')

print("ZIP file created successfully!")

In [ ]:
# Load the pre_train_list and pre_val_list from the ZIP file
pre_train_list = load_from_zip('./processed_data/pre_training_data_img6_r30_n1_random_01.zip', 'pre_train_list.pt')
pre_val_list = load_from_zip('./processed_data/pre_training_data_img6_r30_n1_random_01.zip', 'pre_val_list.pt')


In [ ]:
# Create DataLoader objects for pre-training and pre-validation
pre_train_loader = DataLoader(pre_train_list, batch_size=1, shuffle=True)
pre_val_loader = DataLoader(pre_val_list, batch_size=1, shuffle=False)

In [ ]:
# Run pretraining

in_channels  = 550
hidden_channels_1 = 256
hidden_channels_2 = 33
out_channels= 550

# Pretraining
# Define the device
device =  torch.device('cuda' if torch.cuda.is_available() else 'cpu') #"cpu"

# Create the model
model = GAT4(in_channels, hidden_channels_1,hidden_channels_2, out_channels).to(device) # in_channels is set to 100 as an example. Please replace it with your actual feature size.

# Define loss function and optimizer
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

num_epochs = 300
patience = 8

train(model=model, train_loader=pre_train_loader, val_loader=pre_val_loader, criterion=criterion, num_epochs= num_epochs, patience = patience, optimizer= optimizer,weight_loss = False, model_path="./models/img6_r30_n1_random_01_GAT4_0.001.pt'")

In [ ]:
# Split the pre_val_list into 80% for tune_train and 20% for temporary validation/test
tune_train, temp_val_test = train_test_split(pre_train_list, test_size=0.20, random_state=42)

# Split the temporary validation/test into 50% for tune_val and 50% for tune_test
tune_val, tune_test = train_test_split(temp_val_test, test_size=0.50, random_state=42)

In [ ]:
# Create DataLoader objects for pre-training and pre-validation
tune_train_loader = DataLoader(tune_train, batch_size=1, shuffle=True)
tune_val_loader = DataLoader(tune_val, batch_size=1, shuffle=False)
tune_test_loader = DataLoader(tune_test, batch_size=1, shuffle=False)

In [ ]:
PRE_TRAINED_MODEL_PATH = "./models/img6_r30_n1_random_01_GAT4_0.001_weight.pt"


In [ ]:
# transfer learning

num_classes = 33
in_channels  = 550
hidden_channels_1 = 256
hidden_channels_2 = 33
out_channels= 550

freeze = True
device =  torch.device('cuda' if torch.cuda.is_available() else 'cpu') #"cpu"
pretrained_model = GAT4(in_channels, hidden_channels_1,hidden_channels_2, out_channels)

if freeze:
    for param in pretrained_model.parameters():
        param.requires_grad = False

model = Transferlearn(pretrained_model,out_channels,hidden_channels_1,num_classes).to(device)
model








In [ ]:

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

train_classification(model=model, train_loader=tune_train_loader, val_loader=tune_val_loader, num_epochs=10,criterion=criterion, num_classes=num_classes,optimizer=optimizer)

In [ ]:
from sklearn.metrics import accuracy_score
cell_type_accuracies = test_classification(model, tune_test_loader, criterion, num_classes)
visualize_cell_type_accuracies(cell_type_accuracies)